In [1]:
import json
from collections import OrderedDict
from itertools import islice
from openpyxl import load_workbook

wb = load_workbook("Pagamentos mensais 2021.xlsx", data_only=True)
#sheet = wb.active

#comentarios | notas
#for row in sheet.rows:
#    for cell in row:
#        print(cell.comment)

#tudo da linha 4 ate ultima coluna
#for i in range(4, sheet.max_row+1):
#    row = [cell.value for cell in sheet[i][start_col:end_col+1]]
#    print(row) # list of cell values of this row

#abstraindo para comentarios sheet.rows
#for row in sheet.rows:
#    print(row[1].value)
#    print(row[1].comment)        

cf_list = []

for sheet in wb:
    if sheet.title not in ("todos","listas","query","development view"):
        ano = "2022" if "2022" in sheet.title else "2021"
        for row in islice(sheet.rows, 3, sheet.max_row):
            cf = OrderedDict()

            #variaveis de entrada
            grupos = []
            tags = []
            entrada = "Entrada mes anterior"
            status = row[0].value
            descricao = row[1].value
            notaStr = row[1].comment
            nota = None
            renda = row[6].value
            gastos = row[7].value
            dataPagamento = row[5].value
            data_do_pagamento = None
            data_de_referencia = None
            data_do_evento = None
            instituicao_financeira = row[8].value
            tagsStr = row[9].value

            if (status and descricao != entrada) or (status and sheet.title == "janeiro"):
                
                #data_do_pagamento
                if dataPagamento:
                    data_do_pagamento = str(dataPagamento).replace(" 00:00:00","T00:00:00")
                
                #data_de_referencia
                dia = row[2].value
                mes = row[4].value
                if dia:
                    dia = str(dia).replace(".0", "").zfill(2)
                    mes = str(mes).zfill(2)
                    data_de_referencia = ano+'-'+mes+'-'+dia+'T00:00:00'
                
                if status == "PAGO" or status == "PREVISTO":
                    if gastos:
                        grupos.append("GASTOS")
                    if renda:
                        grupos.append("RENDA")
                    if data_do_pagamento is None:
                        data_do_pagamento = data_de_referencia
                    data_do_evento = data_de_referencia

                if (status == "CONTAS" or status == "TRANSFERINDO") and renda:
                    grupos.append("TRANSFERENCIA")
                    status = "RECEBIDO" if status == "CONTAS" else "PREVISTO"
                    data_do_evento = data_de_referencia
                
                if (status == "CONTAS" or status == "TRANSFERINDO") and gastos:
                    grupos.append("TRANSFERENCIA")
                    status = "ENVIADO" if status == "CONTAS" else "PREVISTO"
                    data_do_evento = data_de_referencia

                if status == "CREDITO" or status == "CREDITANDO":
                    #grupos
                    if status == "CREDITO":
                        grupos.append("CREDITO")
                    else: 
                        grupos.append("CREDITO")
                        grupos.append("GASTOS")
                    #status
                    if status == "CREDITO":
                        status = "PAGO"
                    else:
                        status = "PREVISTO"
                    
                    data_do_evento = data_do_pagamento
                    data_do_pagamento = data_de_referencia            

                #nota
                if notaStr:
                    nota = str(notaStr).replace("Comment: ", "").replace(" by None", "")
                
                #tags
                if tagsStr: 
                    tagsIn = []
                    for tag in tagsStr.split('#'):
                        if tag:
                            tagsIn.append(tag.strip())
                    tags = tagsIn

                #setando null em listas vazias
                if not grupos:
                    grupos = None
                if not tags:
                    tags = None

                #preparando lista de saida
                cf['status'] = status
                cf['descricao'] = descricao
                cf['nota'] = nota
                cf['data_de_referencia'] = data_de_referencia
                cf['data_do_pagamento'] = data_do_pagamento
                cf['data_do_evento'] = data_do_evento
                cf['renda'] = renda
                cf['gastos'] = gastos
                cf['grupos'] = grupos
                cf['instituicao_financeira'] = instituicao_financeira
                cf['tags'] = tags

                cf_list.append(cf)

#https://stackoverflow.com/questions/18337407/saving-utf-8-texts-with-json-dumps-as-utf8-not-as-u-escape-sequence
j = json.dumps(cf_list, indent = 4, default = str, ensure_ascii=False).encode('utf8')
print(j.decode())

#with open('data.json', 'w') as f:
#    f.write(j)

[
    {
        "status": "PAGO",
        "descricao": "Entrada mes anterior",
        "nota": null,
        "data_de_referencia": "2021-01-01T00:00:00",
        "data_do_pagamento": "2021-01-01T00:00:00",
        "data_do_evento": "2021-01-01T00:00:00",
        "renda": 26.78,
        "gastos": null,
        "grupos": [
            "RENDA"
        ],
        "instituicao_financeira": "ITAU",
        "tags": [
            "Recebimentos",
            "Ajuste",
            "Variavel"
        ]
    },
    {
        "status": "PAGO",
        "descricao": "Entrada mes anterior",
        "nota": null,
        "data_de_referencia": "2021-01-01T00:00:00",
        "data_do_pagamento": "2021-01-01T00:00:00",
        "data_do_evento": "2021-01-01T00:00:00",
        "renda": 2.85,
        "gastos": null,
        "grupos": [
            "RENDA"
        ],
        "instituicao_financeira": "INTER",
        "tags": [
            "Recebimentos",
            "Ajuste",
            "Variavel"
        ]
 